### **딥러닝 분류 모델**

Yolo를 활용하여 웨이퍼의 결함 패턴을 검출하고, 웨이퍼의 결함 패턴을 분류하는 모델을 사용합니다. 최적의 분류 모델을 사용하기 위해 NN 모델, 전이학습 모델 중에서 가장 정확도가 높은 모델을 판별한 후, 성능을 개선할 예정입니다.



In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 모델 분석 및 시각화 도구
import os, numpy as np, random
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# 모델 설계
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [ ]:
import os
import numpy as np
from PIL import Image

# 데이터셋 경로
dataset_path = "/content/drive/MyDrive/Dataset"

# 클래스와 라벨 매핑
classes = {"Center": 0, "Donut": 1, "Edge-Loc": 2, "Edge-Ring": 3, "Loc": 4,
           "Near-full": 5, "Random": 6, "Scratch": 7}

# 데이터와 라벨 저장
image_data = []
labels = []

# 폴더를 순회하며 이미지 로드 및 라벨링
for class_name, label in classes.items():
    class_folder = os.path.join(dataset_path, class_name)
    for image_file in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_file)

        # 이미지 로드 (필요에 따라 리사이즈)
        try:
            img = Image.open(image_path).convert("RGB")
            img = img.resize((128, 128))
            image_data.append(np.array(img))
            labels.append(label)
        except Exception as e:
            print(f"이미지 로드 실패: {image_path}, 오류: {e}")

# 배열로 변환
X = np.array(image_data)
y = np.array(labels)

# 라벨 분포 출력
print("라벨 분포:", dict(zip(*np.unique(y, return_counts=True))))


추가 코드: Near-full 패턴의 데이터 증강

In [ ]:
# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 가장 많은 클래스 기준으로 증강
class_counts = dict(zip(*np.unique(y, return_counts=True)))
max_count = max(class_counts.values())

X_aug, y_aug = [], []

for class_label in np.unique(y):
    class_images = X[y == class_label]
    needed = max_count - len(class_images)
    if needed > 0:
        for img in class_images:
            img = img.reshape((1, *img.shape))
            for batch in datagen.flow(img, batch_size=1):
                X_aug.append(batch[0])
                y_aug.append(class_label)
                if len([l for l in y_aug if l == class_label]) >= needed:
                    break

X_bal = np.concatenate([X, np.array(X_aug)], axis=0)
y_bal = np.concatenate([y, np.array(y_aug)], axis=0)
X_bal, y_bal = shuffle(X_bal, y_bal, random_state=42)

print("📊 증강 후 클래스 분포:", dict(zip(*np.unique(y_bal, return_counts=True))))

In [ ]:
# 정규화 (0 ~ 1)
X_bal = X_bal / 255.0
y_bal_cat = to_categorical(y_bal)

X_train, X_test, y_train, y_test = train_test_split(
    X_bal, y_bal_cat, test_size=0.2, random_state=42, stratify=y_bal)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
def visualize_images(X, y, label_map, num=10):
    plt.figure(figsize=(15, 6))
    indices = random.sample(range(len(X)), num)
    for i, idx in enumerate(indices):
        img = X[idx]
        label = np.argmax(y[idx])
        plt.subplot(2, 5, i + 1)
        plt.imshow(img)
        plt.title(label_map[label])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_images(X_train, y_train, classes)

In [ ]:
from collections import Counter

# 클래스별 라벨 카운트 출력
label_counts = Counter(y)
print("Training data label counts:")
for label, count in label_counts.items():
    class_name = "healthy_strawberry" if label == 0 else "infected_strawberry"
    print(f"{class_name}: {count}")

# 데이터셋의 라벨을 샘플로 확인
print("\nSample labels:")
sample_labels = ["healthy_strawberry" if label == 0 else "infected_strawberry" for label in y[:10]]
print(sample_labels)  # 앞 10개 라벨 출력

In [ ]:
# 모델 결과 시각화 함수
def plot_history(history, main_title):
    plt.figure(figsize=(12, 5))

    # 전체 타이틀
    plt.suptitle(main_title, fontsize=20, fontweight='bold')

    # 정확도
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy', fontsize=16)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend()

    # 손실
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend()

    plt.tight_layout(rect=[0, 0, 1, 0.92])  # 전체 타이틀 공간 확보
    plt.show()

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',   # 검증 손실을 모니터링
    patience=10,           # 개선이 없을 때 대기할 에포크 수
    restore_best_weights=True  # 최적의 가중치를 복원
)